## Open Source LLMs

In [1]:
! pip install transformers ctransformers accelerate

In [2]:
! pip install sentencepiece

In [3]:
import torch

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Stable LM

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [6]:
#model_name = "stabilityai/stablelm-base-alpha-3b-v2"

In [7]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code =True,
#     torch_dtype ="auto",
# )

In [8]:
#model.cuda()

In [9]:
#inputs = tokenizer("The weather is always wonderful", return_tensors="pt").to(device)

In [10]:
# tokens = model.generate(
# **inputs,
# max_new_tokens=300,
# temperature=0.75,
# top_p=0.95,
# do_sample=True,
# )

In [11]:
# print(tokenizer.decode(tokens[0], skip_special_tokens=True))

In [12]:
# torch.cuda.empty_cache()

### Orca-Mini-3b

In [13]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [14]:
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
model_path, torch_dtype=torch.float16, device_map='auto',
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
prompt = "Write about ISRO and his achievements"

In [16]:
tokens = tokenizer.encode(prompt)
tokens = torch.LongTensor(tokens).unsqueeze(0)
tokens = tokens.to(device)

In [17]:
instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.2, 'generate_len': 512, 'top_k': 30}

In [19]:
length = len(tokens[0])
with torch.no_grad():
 response = model.generate(
 input_ids=tokens,
 max_length=length+instance['generate_len'],
 top_p=instance['top_p'],
 temperature=instance['temperature'],
 top_k=instance['top_k'],
 use_cache=True,
 do_sample=True,
 )

In [20]:
output = response[0][length:]
result = tokenizer.decode(output, skip_special_tokens=True)
result

".\nISRO (Indian Space Research Organization) is a space agency of India that was established in 1969. It has made many significant achievements in the field of space exploration. Here are some of them:\n1. First Indian satellite - APPLE: The first Indian satellite, APPLE, was launched by ISRO in 1975. It was designed to study the Earth's magnetic field and its variations.\n2. First Indian spacecraft to orbit the Moon - Chandrayaan-1: ISRO launched Chandrayaan-1, the first Indian spacecraft to orbit the Moon, in 2008. The spacecraft was designed to study the Moon's surface and its composition.\n3. Mars Orbiter Mission: ISRO launched the Mars Orbiter Mission (MOM) in 2013, which was the first Indian mission to Mars. The mission included a spacecraft that was designed to orbit the planet and a lander that was designed to touch down on the surface of Mars.\n4. INSAT series of satellites: ISRO has launched several INSAT series of satellites that are used for communication, weather monitori

### GPT4All